In [176]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2
import scipy
import math
import pickle


#Scaling
normilize = True
scale = False

#Spliting features and classifiers
SUBSET_FEATURES = True
SUBMODELS = False

#Showing Each classifier
SEPERATE_CLASSIFIERS = True

#Combining output
NN_COMBINE = True
VOTING_CLASSIFIERS = True


pd.set_option('display.max_columns', None)

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()



sql = "select   fights.f1result, \
                f1.height_inches AS f1_height,\
                f1.reach_inches AS f1_reach, \
                f1.weight_lbs AS f1_weight, \
                f1.strike_offense_per_min AS f1_strike_offense_per_min, \
                f1.strike_offense_accuracy AS f1_strike_offense_accuracy, \
                f1.strike_defense_per_min AS f1_strike_defense_per_min, \
                f1.strike_defense_accuracy AS f1_strike_defense_accuracy, \
                f1.takedowns_per_fight AS f1_takedowns_per_fight, \
                f1.takedowns_accuracy AS f1_takedowns_accuracy, \
                f1.takedowns_defense AS f1_takedowns_defense, \
                f1.submissions_per_fight AS f1_submissions_per_fight, \
                f1.total_fights AS f1_total_fights, \
                f1.association AS f1_association, \
                f1.country AS f1_country, \
                f1.wins AS f1_wins, \
                f1.losses AS f1_losses, \
                f1.total_fights AS f1_total_fights, \
                                                \
                f2.height_inches AS f2_height, \
                f2.reach_inches AS f2_reach, \
                f2.weight_lbs AS f2_weight, \
                f2.strike_offense_per_min AS f2_strike_offense_per_min, \
                f2.strike_offense_accuracy AS f2_strike_offense_accuracy, \
                f2.strike_defense_per_min AS f2_strike_defense_per_min, \
                f2.strike_defense_accuracy AS f2_strike_defense_accuracy, \
                f2.takedowns_per_fight AS f2_takedowns_per_fight, \
                f2.takedowns_accuracy AS f2_takedowns_accuracy, \
                f2.takedowns_defense AS f2_takedowns_defense, \
                f2.submissions_per_fight AS f2_submissions_per_fight, \
                f2.total_fights AS f2_total_fights, \
                f2.association AS f2_association, \
                f2.country AS f2_country, \
                f2.wins AS f2_wins, \
                f2.losses AS f2_losses, \
                f2.total_fights AS f2_total_fights \
                from octagon.fights INNER JOIN octagon.fighters AS f1 ON octagon.fights.f1name = f1.name INNER JOIN octagon.fighters AS f2 ON octagon.fights.f2name = f2.name;"

df = pd.read_sql_query(sql, conn)

test = pd.read_sql_query("select * from octagon.fighters",conn)



# df = pd.read_sql_query("select * from octagon.fights",conn)
df2 = pd.read_sql_query("select * from octagon.fighters",conn)
conn.close()


In [177]:
test.head(n=20)

,fid,name,dob,height_inches,weight_lbs,association,division,country,reach_inches,strike_offense_per_min,strike_offense_accuracy,strike_defense_per_min,strike_defense_accuracy,takedowns_per_fight,takedowns_accuracy,takedowns_defense,submissions_per_fight,wins,losses,draws,total_fights,win_pct,finish_pct
0,230,John Lewis,1969-07-16,72,160,Lewis,Welterweight,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,0
1,16635,Tom Speer,1984-08-20,71,170,None,Welterweight,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,2,0,0
2,95457,Marcio Alexandre Jr.,1989-05-03,73,170,Team Tavares,Welterweight,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,0,3,0,0
3,3069,Mike Thomas Brown,1975-09-08,66,145,American Top Team,Featherweight,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,0,6,33,0
4,354,Kelly Dullanty,1977-07-04,68,155,None,Lightweight,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,0
5,12078,Mizuto Hirota,1981-05-05,67,145,Cave,Featherweight,Japan,67,3.49,45,3.25,53,2.16,68,64,0.0,0,2,1,3,0,0
6,258,Alex Andrade,1974-05-14,71,200,Lion's Den,Light Heavyweight,United States,NaN,0.20,36,2.60,53,0.00,0,25,0.8,0,0,0,0,0,0
7,87012,Cesar Arzamendia,1991-01-31,71,155,Paraguay Combat Team,Featherweight,Paraguay,71,5.37,45,5.20,56,7.32,75,0,2.4,0,0,0,0,0,0
8,132,Matt Andersen,1971-05-29,74,230,None,Heavyweight,USA,NaN,0.00,0,0.00,0,0.00,0,0,0.0,0,0,0,0,0,0
9,60,Andy Anderson,1964-07-16,69,240,None,Heavyweight,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0


### Convert text ones into numeric with normilization

In [178]:


country_names = list((df['f1_country'].append(df['f2_country'])).unique())
df['f1_country'] = df['f1_country'].apply(lambda x: country_names.index(x) / len(country_names))
df['f2_country'] = df['f2_country'].apply(lambda x: country_names.index(x) / len(country_names))


asso_names = list((df['f1_association'].append(df['f2_association'])).unique())
df['f1_association'] = df['f1_association'].apply(lambda x: asso_names.index(x) / len(asso_names))
df['f2_association'] = df['f2_association'].apply(lambda x: asso_names.index(x) / len(asso_names))

# df['f1_association'] = df['f1_association'].apply(lambda x: names.index(x))
# df['f2_association'] = df['f2_association'].apply(lambda x: names.index(x))

df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,win,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.000000,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.343750,0,1,2,3
1,win,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.001645,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.093750,0,15,10,25
2,win,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.003289,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.182566,0,1,2,3
3,win,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004934,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.309211,0,0,2,2
4,win,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.006579,0.016667,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.003289,0,0,1,1


In [179]:
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])
df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,1,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.000000,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.343750,0,1,2,3
1,1,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.001645,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.093750,0,15,10,25
2,1,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.003289,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.182566,0,1,2,3
3,1,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004934,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.309211,0,0,2,2
4,1,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.006579,0.016667,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.003289,0,0,1,1


### Subsetting the features to improve performance

In [180]:
if SUBSET_FEATURES:
    subset = ['f1result','f1_strike_offense_per_min','f1_strike_defense_per_min','f1_association','f1_wins','f1_losses', 
        'f2_strike_offense_per_min','f2_strike_defense_per_min','f2_association','f2_wins','f2_losses']
    df = df[subset]

### Interpolate the data to fill in the NAs to get a better result of the data
It was found that dropna works the best for the highest correclation results, We will revisit this later

In [181]:
df0 = df.interpolate()
df0 = df.dropna()
df0 = df.interpolate(method='spline', order=2)
df = df.interpolate(method='pchip')

df.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,1,2.82,2.46,0.000000,1,3,0.85,2.48,0.343750,1,2
1,1,NaN,NaN,0.001645,4,3,1.79,2.21,0.093750,15,10
2,1,1.89,3.23,0.003289,1,2,2.39,2.37,0.182566,1,2
3,1,0.97,2.87,0.004934,4,3,2.61,3.91,0.309211,0,2
4,1,1.61,2.00,0.006579,4,5,2.55,2.34,0.003289,0,1


## Move the lossers to the end to have data for losing people

In [182]:
df.corr()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
f1result,1.000000,-0.035441,-0.028436,0.019146,-0.001856,-0.025664,-0.025063,-0.035221,0.041615,0.000294,-0.020774
f1_strike_offense_per_min,-0.035441,1.000000,0.350918,0.069149,0.117497,-0.121948,0.217637,0.200056,0.014273,0.060027,0.031935
f1_strike_defense_per_min,-0.028436,0.350918,1.000000,0.074457,-0.154256,0.028993,0.186601,0.137541,0.067362,-0.077992,-0.026982
f1_association,0.019146,0.069149,0.074457,1.000000,-0.372542,-0.346534,0.059530,0.069150,0.178505,-0.186435,-0.172257
f1_wins,-0.001856,0.117497,-0.154256,-0.372542,1.000000,0.539738,-0.004114,-0.113385,-0.230610,0.348939,0.278421
f1_losses,-0.025664,-0.121948,0.028993,-0.346534,0.539738,1.000000,-0.077333,-0.034895,-0.175881,0.167055,0.250639
f2_strike_offense_per_min,-0.025063,0.217637,0.186601,0.059530,-0.004114,-0.077333,1.000000,0.161100,-0.150191,0.270764,0.154747
f2_strike_defense_per_min,-0.035221,0.200056,0.137541,0.069150,-0.113385,-0.034895,0.161100,1.000000,0.138830,-0.215027,-0.102071
f2_association,0.041615,0.014273,0.067362,0.178505,-0.230610,-0.175881,-0.150191,0.138830,1.000000,-0.402357,-0.374376
f2_wins,0.000294,0.060027,-0.077992,-0.186435,0.348939,0.167055,0.270764,-0.215027,-0.402357,1.000000,0.695793


In [183]:
f1_end = math.ceil(len(df.columns) / 2)
df2 = df.copy()
tmp = df2.ix[:,1:f1_end]


df2.ix[:,1:f1_end] = df2.ix[:,f1_end:].values
df2.ix[:,f1_end:] = tmp.values
df2['f1result'] = 0


df2.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,0,0.85,2.48,0.343750,1,2,2.82,2.46,0.000000,1,3
1,0,1.79,2.21,0.093750,15,10,NaN,NaN,0.001645,4,3
2,0,2.39,2.37,0.182566,1,2,1.89,3.23,0.003289,1,2
3,0,2.61,3.91,0.309211,0,2,0.97,2.87,0.004934,4,3
4,0,2.55,2.34,0.003289,0,1,1.61,2.00,0.006579,4,5


### Combine the two dataframes and randomize

In [184]:
df = df.append(df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,0,3.22,2.61,0.245066,5,4,1.19,1.78,0.138158,6,4
1,1,4.03,3.33,0.062500,7,2,NaN,NaN,0.233553,8,5
2,0,1.12,1.96,0.108553,4,5,2.32,2.14,0.213816,5,3
3,0,3.78,3.91,0.955592,0,2,NaN,NaN,0.554276,2,1
4,1,3.67,3.99,0.631579,3,1,NaN,NaN,0.588816,1,2


In [185]:
#Pull the labels out of the dataset
y = df['f1result']
del df['f1result']

X = df

In [186]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split
#MLP Classifer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.feature_selection import RFE

if normilize:
    X = preprocessing.normalize(X, norm='l2')
    
if scale:
    X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state = 1)

names = [
         "MLP",
         "Naive Bayes",
         "KNN",
         "SVM Linear",
         "SVM gamma",
         "Decsion Tree", 
         "Random Forest",
         "adaBoost"
        ]

clfs = [
        MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=400),
        GaussianNB(),
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025,probability=True),
        SVC(gamma=2, C=1,probability=True),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        AdaBoostClassifier()
       ]

if SUBMODELS and SUBSET_FEATURES:
    #feats = [0,2,3]
    feats = [0,2,3,4,5,6,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]
elif SUBMODELS:
    feat = [3,5,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]

#predict_proba to get the probabilities 
    
    #All Features
# SVC- Linear, DTree and AdaBoost are the best especially with Voter
 
#Subset
# MLP, KNN, SVM Linear 
# Normilization Good 
    
    
#13, 15, 16, 30, 
# print("NBC Accuracy of the model is: %.2f%%" % (accuracy_score(nbc_preds,y_test) * 100))
# print("MLP Accuracy of the model is: %.2f%%" % (accuracy_score(mlp_preds,y_test) * 100))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Finding the importance of each feature using Recursion

In [ ]:
if False:
    f1_end = int(len(df.columns) / 2)
    indexs = np.array([i for i in range(f1_end)])
    selector = RFE(clfs[3], 14, step=1)
    selector = selector.fit(X_train, y_train)
    print(selector.support_)
    print(indexs[selector.support_[:f1_end]])
    print(indexs[selector.support_[f1_end:]])
#

## Looking at each Classifier separatly 

In [ ]:
if SEPERATE_CLASSIFIERS:
    
    pred_list_train = []
    pred_list_test = []
    
    for index, clf in enumerate(clfs):
        clf.fit(X_train,y_train)
        preds = clf.predict(X_test)
        
        preds_train = np.amax(clf.predict_proba(X_train),axis=1)
        preds_test = np.amax(clf.predict_proba(X_test),axis=1)
        
        pred_list_train.append(preds_train)
        pred_list_test.append(preds_test)
        #print(names[index] + " Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))
        print("%.2f" % (accuracy_score(preds,y_test) * 100))

      

## Using Voting for Combination

In [ ]:
if VOTING_CLASSIFIERS:
    clf_vote = VotingClassifier(estimators=list(zip(names,clfs)),voting='soft')
    clf_vote.fit(X_train,y_train)
    preds = clf_vote.predict(X_test)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))


### Testing Combination of Classifiers using NN - DOES NOT WORK GOOD

In [ ]:
if NN_COMBINE:
    combine_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    combine_clf.fit(np.array(pred_list_train).T,y_train)
    preds = combine_clf.predict(np.array(pred_list_test).T)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))



In [187]:

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()

sql = "select fid, \
            height_inches, \
            reach_inches, \
            weight_lbs, \
            strike_offense_per_min, \
            strike_offense_accuracy, \
            strike_defense_per_min, \
            strike_defense_accuracy, \
            takedowns_per_fight, \
            takedowns_accuracy, \
            takedowns_defense, \
            submissions_per_fight, \
            total_fights, \
            association, \
            country, \
            wins, \
            losses, \
            total_fights FROM octagon.fighters"

#16635
#230

#df1 = pd.read_sql_query(sql + str(16635), conn)
df = pd.read_sql_query("select * from octagon.fighters", conn)

conn.close()
df[df['fid'] == 12078]

,fid,name,dob,height_inches,weight_lbs,association,division,country,reach_inches,strike_offense_per_min,strike_offense_accuracy,strike_defense_per_min,strike_defense_accuracy,takedowns_per_fight,takedowns_accuracy,takedowns_defense,submissions_per_fight,wins,losses,draws,total_fights,win_pct,finish_pct
5,12078,Mizuto Hirota,1981-05-05,67,145,Cave,Featherweight,Japan,67,3.49,45,3.25,53,2.16,68,64,0,0,2,1,3,0,0


In [194]:
#Figure out good interpolator without having to load the entire database each time
df = df.interpolate()

# f1 = df[df['fid'] == 230].drop('fid',axis=1).values
# f2 = df[df['fid'] == 16635].drop('fid',axis=1).values

# np.append(f1,f2)

df[df['fid']


SyntaxError: unexpected EOF while parsing (<ipython-input-194-453459f79e56>, line 9)